In [1]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

We're running Colab


In [2]:
if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/drive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive/Thesis_coding_with_IAE"
  
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root

Colab: mounting Google drive on  /content/drive
Mounted at /content/drive

Colab: Changing directory to  /content/drive/My Drive/Thesis_coding_with_IAE
/content/drive/My Drive/Thesis_coding_with_IAE


In [17]:
import pandas as pd
import scipy
import scipy.spatial
import networkx as nx
import numpy as np
from collections import deque
import collections
import pickle

# for plotting graphs
import matplotlib.pyplot as plt

from CommonDefs import Point, Edge 
from TwoDimConvexHull import TwoDimConvexHull, PrintTwoDimConvexHull
from Utils import nearestKNeighboursOfEdgeMidPt, checkTriangleForDelaunayCriteria
from Graph import Graph
from DTEnv import DTEnv
from DTDQNAgent import DTDQNAgent
from DTDoubleDQNAgent import DTDoubleDQNAgent

In [18]:
pd.options.display.max_columns = None

In [19]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
df = pd.read_csv("/content/drive/My Drive/Data/DT_10_sorted.txt", sep=",", header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56
0,0.018435,0.670358,0.101245,0.711891,0.207086,0.100722,0.238450,0.164358,0.321183,0.967308,0.399677,0.449653,0.417594,0.010150,0.920121,0.307752,0.939591,0.158107,0.965258,0.480327,output,1,2,5,1,2,6,1,3,4,3,4,7,2,5,6,1,4,6,4,6,7,5,6,10,6,7,8,6,8,10,7,8,9,8,9,10
1,0.108702,0.075571,0.154823,0.495594,0.327407,0.047181,0.529001,0.658846,0.641060,0.224470,0.645282,0.274440,0.666990,0.731345,0.718659,0.101208,0.808132,0.025225,0.993713,0.533967,output,1,2,3,2,3,6,2,4,6,2,4,7,4,6,7,3,5,8,3,5,6,5,6,8,3,8,9,6,8,10,6,7,10,8,9,10
2,0.048977,0.558356,0.090114,0.358360,0.414100,0.802224,0.457654,0.972640,0.740652,0.717631,0.851315,0.117623,0.873359,0.345104,0.905256,0.762454,0.931034,0.006254,0.940295,0.553189,output,1,2,3,1,3,4,2,3,7,3,4,5,3,5,7,4,5,8,2,6,7,2,6,9,5,8,10,5,7,10,6,7,9,7,9,10
3,0.048994,0.994292,0.134057,0.220541,0.169525,0.608264,0.177805,0.178609,0.429897,0.374512,0.537223,0.829864,0.561325,0.438124,0.642534,0.113659,0.717887,0.584209,0.851976,0.429216,output,1,2,3,2,4,5,1,3,6,2,3,5,4,5,8,3,6,7,3,5,7,5,7,8,6,7,9,7,8,10,7,9,10,6,9,10
4,0.048897,0.363199,0.093488,0.236462,0.128710,0.457687,0.142823,0.809028,0.233381,0.174737,0.288991,0.625024,0.395214,0.208379,0.567808,0.426278,0.880467,0.112534,0.962764,0.756392,output,1,2,3,1,3,4,2,3,5,3,4,6,3,6,7,3,5,7,4,6,10,5,7,9,6,7,8,6,8,10,7,8,9,8,9,10


In [21]:
df.shape

(1243, 57)

In [22]:
pointData = {}
optimalDT = {}
for i in range(len(df)):
    pointId = 1
    points = []
    for j in range(0 , len(df.columns), 2):
        if df.loc[i, j] == "output":
            triList = []
            for k in range(j+1, len(df.columns), 3):
                tri=[df.loc[i, k], df.loc[i, k+1], df.loc[i, k+2]]
                tri = sorted(tri)
                triList.append(tri)
            optimalDT[i] = triList
            break
        else:
            points.append(Point(pointId, df.loc[i, j], df.loc[i, j + 1]))
            pointId = pointId + 1
    pointData[i] = points

In [23]:
# Initialising the environment
env = DTEnv()
agent = DTDoubleDQNAgent(env)

In [26]:
initializeModel = True

for i in range(15):     

    score = 0
    env.reset()
    env.initialize(pointData[i])  

    if initializeModel:
        agent.initializeModelForValidation(env, 'DDQN_Delaunay.h5')
        initializeModel = False

    _, state = env.getStartState()
    terminal_state = False

    while not terminal_state:

        #Get the free edge from the list
        edgeToProcess = env.getEdgesToProcess()[0]

        action = agent.get_action_for_validation(state, edgeToProcess)            

        '''if not action:
            break'''

        #print('edgeToProcess: {}-{}'.format(edgeToProcess[0], edgeToProcess[1]))
        #print('action chosen: {}'.format(env.getNodeIdFromPosAttr( action[4], action[5])))

        reward = env.getReward(edgeToProcess, action)        
        next_state = env.getNextState(edgeToProcess, action)
        env.removeProcessedEdge(edgeToProcess)
        terminal_state = env.isTerminalState()

        score += reward
        state = next_state

    if terminal_state:
        print('generated triangles: {}'.format(env.getGeneratedTriangles()))


      

    
    

generated triangles: [[1, 2, 5], [1, 2, 8], [2, 5, 7], [1, 7, 8], [2, 3, 8], [2, 3, 7], [5, 6, 7], [1, 5, 7], [3, 7, 8], [2, 3, 5], [3, 8, 10], [5, 6, 10], [6, 7, 10], [3, 5, 7], [1, 3, 10], [5, 8, 10], [1, 7, 10], [1, 5, 10], [1, 5, 8]]
generated triangles: [[1, 2, 3], [2, 3, 4], [2, 4, 7], [3, 4, 9], [4, 6, 7], [4, 9, 10], [3, 4, 6], [6, 7, 8], [4, 7, 10], [3, 6, 9], [5, 6, 8], [7, 8, 10], [5, 6, 9], [5, 6, 7], [8, 9, 10], [5, 8, 9]]
generated triangles: [[4, 5, 8], [3, 4, 5], [5, 8, 9], [1, 3, 4], [3, 5, 7], [5, 6, 9], [8, 9, 10], [1, 3, 5], [1, 3, 7], [4, 5, 7], [4, 5, 6], [1, 6, 9], [1, 5, 8], [1, 4, 7], [1, 4, 6], [1, 4, 9]]
generated triangles: [[1, 2, 3], [1, 3, 6], [2, 3, 9], [3, 6, 9], [2, 7, 9], [3, 7, 9], [6, 9, 10], [2, 5, 7], [7, 9, 10], [3, 5, 7], [5, 7, 8], [7, 8, 10], [1, 3, 5], [3, 5, 8]]
generated triangles: [[1, 2, 7], [1, 7, 8], [2, 5, 7], [1, 3, 8], [7, 8, 9], [5, 7, 9], [1, 3, 4], [3, 4, 8], [8, 9, 10], [3, 4, 7], [4, 8, 9], [4, 8, 10], [3, 7, 9], [4, 7, 9], [4, 